# Problem 2: Political Network Connections

In this problem, you will analyze the network connections and strength between all persons and organizations in the *Trump World* using a combination of hash tables (i.e., dictionaries) and pandas dataframe.  

## The dataset

The dataset for this problem is built from public records, news reports, and other sources on the Trump family, his Cabinet picks, and top advisers - more than 1,500 people and organizations altogether. 

Each row represents a connection between a person and an organization (e.g., The Trump Organization Inc. and Donald J. Trump), a person and another person (e.g., Donald J. Trump and Linda McMahon), or two organizations (e.g., Bedford Hills Corp. and Seven Springs LLC).

Source: https://www.buzzfeednews.com/article/johntemplon/help-us-map-trumpworld

Before starting, please run the following cell to set up the environment and import the data to `Network`.

In [3]:
import math
import pandas as pd
import numpy as np
from collections import defaultdict
Network = pd.read_csv("../resource/asnlib/publicdata/network.csv",encoding='latin-1' )
assert len(Network) == 3380
Network.head()

,Entity A Type,Entity A,Entity B Type,Entity B,Connection_Strength
0,Organization,4 SHADOW TREE LANE MEMBER CORP.,Organization,4 SHADOW TREE LANE LLC,0.469155
1,Organization,40 WALL DEVELOPMENT ASSOCIATES LLC,Organization,40 WALL STREET LLC,0.035480
2,Organization,40 WALL STREET LLC,Organization,40 WALL STREET COMMERCIAL LLC,0.177874
3,Organization,40 WALL STREET MEMBER CORP.,Organization,40 WALL STREET LLC,0.236508
4,Organization,401 MEZZ VENTURE LLC,Organization,401 NORTH WABASH VENTURE LLC,0.169532


**Exercise 0** (1 points). Create a subset of the data frame named `Network_sub`, keeping only records where `Entity B` contains the keyword "TRUMP" (not case sensitive).

In [47]:
Network_sub = Network.copy()
Network_sub = Network_sub[Network_sub['Entity B'].str.contains('TRUMP', regex=True)]


In [48]:
# Test cell: `test_subset`

assert type(Network_sub)==pd.DataFrame, "Your subset is not a panda dataframe"
assert list(Network_sub)==['Entity A Type','Entity A','Entity B Type','Entity B','Connection_Strength'], "Your subset columns are not consistent with the master dataset"
assert len(Network_sub)==648, "The length of your subset is not correct"

test = Network_sub.sort_values(by='Connection_Strength')
test.reset_index(drop=True, inplace=True)
assert test.loc[0,'Connection_Strength']==0.001315204
assert test.loc[200,'Connection_Strength']==0.312599997
assert test.loc[400,'Connection_Strength']==0.610184514
assert test.loc[647,'Connection_Strength']==0.996641965

print("\n(Passed.)")


(Passed.)


Now, let's take a look at part of the `Network_sub` data.

In [71]:
#Network_sub[25:36]
Network_sub[(Network_sub['Entity A'] == 'DONALD J. TRUMP') & (Network_sub['Entity B Type'] == 'Person')]

,Entity A Type,Entity A,Entity B Type,Entity B,Connection_Strength
584,Person,DONALD J. TRUMP,Person,DONALD TRUMP JR.,0.453991
674,Person,DONALD J. TRUMP,Person,ERIC TRUMP,0.468002
709,Person,DONALD J. TRUMP,Person,IVANKA TRUMP,0.773875
758,Person,DONALD J. TRUMP,Person,MARYANNE TRUMP BARRY,0.330120
761,Person,DONALD J. TRUMP,Person,MELANIA TRUMP,0.517144


**Exercise 1** (4 points). Write a function 

```python
def Connection_Strength(Network_sub, Entity_B_Type)
```

that takes two inputs

1. `Network_sub` is the dataset you get from exercise 0
2. `Entity_B_Type` can take two values: either `Person` or `Organization`

and for every entity A that is connected to entity B, based on the type of entity B, returns a nested dictionary (i.e. dictionary of dictionaries) of the form:

```python 
{Entity A: {Entity B: Connection_Strength, Entity B: Connection_Strength}, ... }```

For example: for entity A that is connected to entity B of type person, the function will return something like the following: 

```python
{'DONALD J. TRUMP': {'DONALD TRUMP JR.': 0.453990548,
  'ERIC TRUMP': 0.468002101,
  'IVANKA TRUMP': 0.773874808,
  'MARYANNE TRUMP BARRY': 0.330120053,
  'MELANIA TRUMP': 0.5171444000000001},
 'DONALD J. TRUMP FOR PRESIDENT, INC.': {'DONALD J. TRUMP': 0.377887355},
 'DONALD TRUMP JR.': {'ERIC TRUMP': 0.405052388, 'VANESSA TRUMP': 0.025756815},
 'GRACE MURDOCH': {'IVANKA TRUMP': 0.966637541},
 'IVANKA M. TRUMP BUSINESS TRUST': {'IVANKA TRUMP': 0.141785871}, ...}```

In [72]:
def Connection_Strength(Network_sub, Entity_B_Type):
    assert type(Entity_B_Type) == str
    assert Entity_B_Type in ['Person', 'Organization']
    
    P = {}
    

    for i in set(Network_sub['Entity A'][Network_sub['Entity B Type'] == Entity_B_Type]):
        
        df = Network_sub[(Network_sub['Entity A'] == i) & (Network_sub['Entity B Type'] == Entity_B_Type)]     
        J = {}
        
        for j in set(df['Entity B']):
            df1 = df[Network_sub['Entity B'] == j]
            J.update({j:df1['Connection_Strength'].iloc[0]})
        
        P.update({i:J})
                    
    return P
        
    
P = Connection_Strength(Network_sub, 'Person')

P['DONALD J. TRUMP']


#len(set(Network_sub['Entity A'][Network_sub['Entity B Type'] == 'Person']))

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


{'DONALD TRUMP JR.': 0.453990548,
 'IVANKA TRUMP': 0.773874808,
 'ERIC TRUMP': 0.468002101,
 'MARYANNE TRUMP BARRY': 0.330120053,
 'MELANIA TRUMP': 0.5171444000000001}

In [73]:
# Test Cell: `Connection_Strength`

# Create a dictonary 'Person' for entity B of type person
Person = Connection_Strength(Network_sub, 'Person')
# Create a dictionary 'Organization' for entity B of type organization
Organization = Connection_Strength(Network_sub, 'Organization')

assert type(Person)==dict or defaultdict, "Your function does not return a dictionary"
assert len(Person)==17, "Your result is wrong for entity B of type person"
assert len(Organization)==296, "Your result is wrong for entity B of type organization"

assert Person['DONALD J. TRUMP']=={'DONALD TRUMP JR.': 0.453990548,'ERIC TRUMP': 0.468002101,'IVANKA TRUMP': 0.773874808,
  'MARYANNE TRUMP BARRY': 0.330120053,'MELANIA TRUMP': 0.5171444000000001}, "Wrong result"
assert Person['DONALD J. TRUMP FOR PRESIDENT, INC.']=={'DONALD J. TRUMP': 0.377887355}, "Wrong result"
assert Person['WENDI DENG MURDOCH']=={'IVANKA TRUMP': 0.669636181}, "Wrong result"

assert Organization['401 MEZZ VENTURE LLC']=={'TRUMP CHICAGO RETAIL LLC': 0.85298544}, "Wrong result"
assert Organization['ACE ENTERTAINMENT HOLDINGS INC']=={'TRUMP CASINOS INC.': 0.202484568,'TRUMP TAJ MAHAL INC.': 0.48784823299999996}, "Wrong result"
assert Organization['ANDREW JOBLON']=={'THE ERIC TRUMP FOUNDATION': 0.629688777}, "Wrong result"

print("\n(Passed.)")

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  



(Passed.)


**Exercise 2** (1 point). For the dictionary `Organization` **created in the above test cell**, create another dictionary `Organization_avg` which for every entity A gives the average connection strength (i.e., the average of nested dictionary values). `Organization_avg` should be in the following form:
```python
{Entity A: avg_Connection_Strength, Entity A: avg_Connection_Strength, ... }```


In [79]:
Organization_avg = {}

for key, value in Organization.items():
    vals = []
    for key2, value2 in value.items():
        vals.append(value2)
    avg = sum(vals) / len(vals)
    Organization_avg.update({key:avg})
    
Organization_avg   
        


{'RICK GATES': 0.059203658,
 'TRUMP OLD POST OFFICE MEMBER CORP.': 0.488701118,
 'BILL STEPIEN': 0.43057139200000005,
 'RUSSELL FLICKER': 0.54442625,
 'TRUMP PARK AVENUE ACQUISITION LLC': 0.19060126300000002,
 'THC CHINA DEVELOPMENT MANAGEMENT CORP.': 0.012192929,
 'IT OPERATIONS MANAGING MEMBER CORP': 0.6276018636666667,
 'LARRY GLICK': 0.826571173,
 'SENTIENT JETS MEMBER CORP.': 0.449202121,
 'HOPE HICKS': 0.43825646050000006,
 'TRUMP PLAZA ASSOCIATES LLC': 0.5092408589999999,
 'TRUMP PRODUCTIONS MANAGING MEMBER, INC.': 0.46170153799999997,
 'TIHM MEMBER CORP.': 0.460458405,
 'JASON D. GREENBLATT': 0.6174344325000001,
 'TRUMP WORLD PRODUCTIONS MANAGER CORP.': 0.996641965,
 'TRUMP MARKS EGYPT CORP.': 0.158861101,
 'T INTERNATIONAL REALTY LLC': 0.37724370399999996,
 'THE TRUMP ENTREPRENEUR INITIATIVE LLC (NY DOMESTIC)': 0.3195141455,
 'MATTHEW CALAMARI, JR.': 0.7601921509999999,
 'MICHAEL SEXTON': 0.511707929,
 'TRUMP GOLF COCO BEACH MEMBER CORP.': 0.576517368,
 'TRUMP MARKS JERSEY CIT

In [80]:
# Test Cell: `Organization_avg`
assert type(Organization_avg)==dict or defaultdict, "Organization_avg is not a dictionary"
assert len(Organization_avg)==len(Organization)

for k_, v_ in {'401 MEZZ VENTURE LLC': 0.85298544,
               'DJT HOLDINGS LLC': 0.5855800477222223,
               'DONALD J. TRUMP': 0.4878277050144927,
               'JAMES BURNHAM': 0.187474088}.items():
    print(k_, Organization_avg[k_], v_)
    assert math.isclose(Organization_avg[k_], v_, rel_tol=4e-15*len(Organization[k_])), \
           "Wrong result for '{}': Expected {}, got {}".format(k_, v_, Organization_avg[k_])

print("\n(Passed.)")

401 MEZZ VENTURE LLC 0.85298544 0.85298544
DJT HOLDINGS LLC 0.5855800477222223 0.5855800477222223
DONALD J. TRUMP 0.48782770501449263 0.4878277050144927
JAMES BURNHAM 0.187474088 0.187474088

(Passed.)


**Exercise 3** (4 points). Based on the `Organization_avg` dictionary you just created, determine which organizations have an average connection strength that is strictly greater than a given threshold, `THRESHOLD` (defined in the code cell below). Then, create a new data frame named `Network_strong` that has a subset of the rows of `Network_sub` whose `Entity A` values match these organizations **and** whose `"Entity B Type"` equals `"Organization"`.

In [91]:
THRESHOLD = 0.5
new = {k: v for k, v in Organization_avg.items() if v > .5}

new_orgs = list(new.keys())



Network_strong = Network_sub[Network_sub['Entity A'].isin(new_orgs)]

Network_strong = Network_strong[Network_strong['Entity B Type'] == 'Organization']

Network_strong

,Entity A Type,Entity A,Entity B Type,Entity B,Connection_Strength
5,Organization,401 MEZZ VENTURE LLC,Organization,TRUMP CHICAGO RETAIL LLC,0.852985
6,Organization,401 NORTH WABASH VENTURE LLC,Organization,TRUMP COMMERCIAL CHICAGO LLC,0.207837
7,Organization,401 NORTH WABASH VENTURE LLC,Organization,TRUMP PAYROLL CHICAGO LLC,0.910172
20,Organization,AETOS SA,Organization,TRUMP MARKS PUNTA DEL ESTE LLC,0.516288
25,Person,ALAN GARTEN,Organization,"THE TRUMP ORGANIZATION, INC.",0.569394
49,Person,ALLEN WEISSELBERG,Organization,THE DONALD J. TRUMP FOUNDATION,0.968860
50,Person,ALLEN WEISSELBERG,Organization,THE DONALD J. TRUMP REVOCABLE TRUST,0.528962
51,Person,ALLEN WEISSELBERG,Organization,"THE TRUMP ORGANIZATION, INC.",0.580072
53,Person,ALLEN WEISSELBERG,Organization,TRUMP INTERNATIONAL GOLF CLUB LC,0.267505
54,Person,ALLEN WEISSELBERG,Organization,TRUMP INTERNATIONAL GOLF CLUB SCOTLAND LIMITED,0.800504


In [92]:
# Test Cell: `Network_strong`
assert type(Network_strong)==pd.DataFrame, "Network_strong is not a panda dataframe"
assert list(Network_strong)==['Entity A Type','Entity A','Entity B Type','Entity B','Connection_Strength'], "Your Network_strong columns are not consistent with the master dataset"
assert len(Network_strong)==194, "The length of your Network_strong is not correct. Correct length should be 194."
test2 = Network_strong.sort_values(by='Connection_Strength')
test2.reset_index(drop=True, inplace=True)
assert math.isclose(test2.loc[0, 'Connection_Strength'], 0.039889119, rel_tol=1e-13)
assert math.isclose(test2.loc[100, 'Connection_Strength'], 0.744171895, rel_tol=1e-13)
assert math.isclose(test2.loc[193, 'Connection_Strength'], 0.996641965, rel_tol=1e-13)

print("\n(Passed.)")


(Passed.)


**Fin!** Remember to test your solutions by running them as the autograder will: restart the kernel and run all cells from "top-to-bottom." Also remember to submit to the autograder; otherwise, you will **not** get credit for your hard work!